<a href="https://colab.research.google.com/github/vanessabrolim/comfort_analysis/blob/master/train_trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install -U mlens

    100% |████████████████████████████████| 235kB 11.8MB/s 


In [2]:
import mlens
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
from scipy import stats
from geopy.distance import vincenty
from sklearn import tree
from sklearn.impute import SimpleImputer
from google.colab import drive
drive.mount('/content/drive',  force_remount=True)

[MLENS] backend: threading


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
df_train = pd.read_pickle('drive/My Drive/Udesc/train.pkl')
df_test = pd.read_pickle('drive/My Drive/Udesc/test.pkl')

In [4]:
df_train.head()

,date_time,engine_rpm,engine_load,intake_manifold_pressure,accelerator_pedal,speed,giroscope_x,giroscope_y,giroscope_z,accelerometer_x_avg,accelerometer_y_avg,accelerometer_z_avg,accelerometer_x_std,accelerometer_y_std,accelerometer_z_std,comfort_x,comfort_y,comfort_z,comfort_geral
14767,2017-09-18 09:47:31.001,NaN,NaN,NaN,NaN,23.223407,-2.72,3.59,-0.34,1.016963,-0.059846,0.120445,0.194853,0.155698,0.172987,2.0,2.0,2.0,3.0
803,2017-09-15 20:18:50.802,1034.423358,55.391512,111.343066,92.270318,3.223987,-0.77,-0.31,-4.57,1.005749,-0.099241,0.110642,0.217798,0.205179,0.219697,4.0,4.0,4.0,4.0
5730,2017-09-16 12:09:42.996,1487.816867,9.590048,118.028169,21.461805,22.807823,-2.05,0.98,4.51,1.024716,-0.063518,0.128107,0.109880,0.089158,0.083585,3.0,4.0,4.0,3.0
7048,2017-09-16 17:12:23.496,NaN,NaN,NaN,NaN,8.774839,-2.20,0.19,-2.51,1.015846,-0.079579,0.139395,0.035262,0.018842,0.042641,3.0,3.0,3.0,3.0
1406,2017-09-15 20:36:53.802,NaN,NaN,NaN,NaN,13.601787,-1.16,-0.10,-2.03,1.012846,-0.077933,0.137599,0.048181,0.030723,0.042666,4.0,4.0,4.0,4.0


In [0]:
df_train_clean = df_train[['engine_rpm', 'engine_load', 'intake_manifold_pressure', 'accelerator_pedal', 'speed', 'giroscope_x', 'giroscope_y', 'giroscope_z','accelerometer_x_avg', 'accelerometer_y_avg', 'accelerometer_z_avg', 'accelerometer_x_std', 'accelerometer_y_std', 'accelerometer_y_std']]

In [6]:
df_train_clean.head()

,engine_rpm,engine_load,intake_manifold_pressure,accelerator_pedal,speed,giroscope_x,giroscope_y,giroscope_z,accelerometer_x_avg,accelerometer_y_avg,accelerometer_z_avg,accelerometer_x_std,accelerometer_y_std,accelerometer_y_std
14767,NaN,NaN,NaN,NaN,23.223407,-2.72,3.59,-0.34,1.016963,-0.059846,0.120445,0.194853,0.155698,0.155698
803,1034.423358,55.391512,111.343066,92.270318,3.223987,-0.77,-0.31,-4.57,1.005749,-0.099241,0.110642,0.217798,0.205179,0.205179
5730,1487.816867,9.590048,118.028169,21.461805,22.807823,-2.05,0.98,4.51,1.024716,-0.063518,0.128107,0.109880,0.089158,0.089158
7048,NaN,NaN,NaN,NaN,8.774839,-2.20,0.19,-2.51,1.015846,-0.079579,0.139395,0.035262,0.018842,0.018842
1406,NaN,NaN,NaN,NaN,13.601787,-1.16,-0.10,-2.03,1.012846,-0.077933,0.137599,0.048181,0.030723,0.030723


In [0]:
list_train = df_train_clean.values.tolist()

In [0]:
df_label_x = df_train[['comfort_x']]
df_label_y = df_train[['comfort_y']]
df_label_z = df_train[['comfort_z']]

In [0]:
list_label_x = df_label_x.values.tolist()
list_label_y = df_label_x.values.tolist()
list_label_z = df_label_x.values.tolist()

In [0]:
df_test_clean = df_test[['engine_rpm', 'engine_load', 'intake_manifold_pressure', 'accelerator_pedal', 'speed', 'giroscope_x', 'giroscope_y', 'giroscope_z','accelerometer_x_avg', 'accelerometer_y_avg', 'accelerometer_z_avg', 'accelerometer_x_std', 'accelerometer_y_std', 'accelerometer_y_std']]
list_test = df_test_clean.values.tolist()
imp_test = SimpleImputer(missing_values=np.nan, strategy='mean')
imp_test = imp_test.fit(list_test) 
test = imp_test.transform(list_test)

In [0]:
df_label_x = df_test[['comfort_x']]
df_label_y = df_test[['comfort_y']]
df_label_z = df_test[['comfort_z']]
list_test_label_x = df_label_x.values.tolist()
list_test_label_y = df_label_x.values.tolist()
list_test_label_z = df_label_x.values.tolist()

In [13]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp = imp.fit(list_train) 
train = imp.transform(list_train)
print(train)

[[1.23348670e+03 2.62271083e+01 1.36271720e+02 ... 1.94852955e-01
  1.55698071e-01 1.55698071e-01]
 [1.03442336e+03 5.53915122e+01 1.11343066e+02 ... 2.17798303e-01
  2.05179151e-01 2.05179151e-01]
 [1.48781687e+03 9.59004819e+00 1.18028169e+02 ... 1.09879545e-01
  8.91575706e-02 8.91575706e-02]
 ...
 [1.56600000e+03 3.64902217e+01 1.85135440e+02 ... 1.53917975e-01
  1.26367208e-01 1.26367208e-01]
 [1.23348670e+03 2.62271083e+01 1.36271720e+02 ... 3.46172426e-03
  3.38105752e-03 3.38105752e-03]
 [1.42448341e+03 4.34964286e+01 1.53469194e+02 ... 1.68314290e-01
  1.53075422e-01 1.53075422e-01]]


In [46]:
tree_x = tree.DecisionTreeClassifier(criterion = 'entropy')
tree_x = tree_x.fit(train, list_label_x)
tree_x.score(test, list_test_label_x)

0.7122406890695135

In [47]:
tree_y = tree.DecisionTreeClassifier(criterion = 'entropy')
tree_y = tree_y.fit(train, list_label_y)
tree_y.score(test, list_test_label_y)

0.7096930729103482

In [48]:
tree_z = tree.DecisionTreeClassifier(criterion = 'entropy')
tree_z = tree_x.fit(train, list_label_z)
tree_z.score(test, list_test_label_z)

0.7147883052286789

In [41]:
tree_2 = tree.ExtraTreeClassifier(criterion = 'entropy')
tree_2 = tree_2.fit(train, list_label_x)
tree_2.score(test, list_test_label_x)

0.8164503214848963

In [42]:
tree_2y = tree.ExtraTreeClassifier(criterion = 'entropy')
tree_2y = tree_2y.fit(train, list_label_y)
tree_2y.score(test, list_test_label_y)

0.821545553803227

In [43]:
tree_2z = tree.ExtraTreeClassifier(criterion = 'entropy')
tree_2z = tree_2z.fit(train, list_label_z)
tree_2z.score(test, list_test_label_z)

0.8159650612641028